In [10]:
from PIL import Image
import json 
import numpy as np
import os 
from website.model.model import get_model
import pickle

from sklearn.model_selection import train_test_split

In [5]:
def read_data(folder, n_per_class = 10_000):
    data = []
    labels = []
    label_mapping = {}
    files = os.listdir(folder)
    for file in files:
        label = len(label_mapping)
        print(file)
        label_mapping[label] = file.split("_")[-1].split(".")[0]
        tmp_data = np.load(os.path.join(folder,file), encoding='latin1', allow_pickle=True)
        for data_point in tmp_data[:n_per_class]:
            data.append(np.array(data_point).reshape((28,28)))
            labels.append(label)
    
    return np.array(data), np.array(labels) , label_mapping


In [6]:
# You can get data by downloading some files from here and put them in the data folder
#https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/numpy_bitmap;tab=objects?prefix=&forceOnObjectsSortingFiltering=false

data, labels, label_mapping = read_data("Data", n_per_class=20_000)

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

full_numpy_bitmap_ant.npy
full_numpy_bitmap_apple.npy
full_numpy_bitmap_bat.npy
full_numpy_bitmap_bee.npy
full_numpy_bitmap_broccoli.npy
full_numpy_bitmap_broom.npy
full_numpy_bitmap_butterfly.npy
full_numpy_bitmap_cat.npy
full_numpy_bitmap_crocodile.npy


In [7]:
model = get_model(len(label_mapping))

In [8]:
x_train_reshaped = x_train.reshape(-1, 28, 28, 1)

# Normalize the data values to range [0, 1]
x_train_normalized = x_train_reshaped / 255.0

x_val_reshaped = x_val.reshape(-1, 28, 28, 1)

# Normalize the data values to range [0, 1]
x_val_normalized = x_val_reshaped / 255.0


# Train the model
history = model.fit(x_train_normalized, y_train, epochs=2, batch_size=256, validation_data=(x_val_normalized, y_val))


Epoch 1/2
563/563 [==============================] - 47s 82ms/step - loss: 0.6313 - accuracy: 0.8023 - val_loss: 0.4265 - val_accuracy: 0.8683
Epoch 2/2
563/563 [==============================] - 52s 92ms/step - loss: 0.3874 - accuracy: 0.8817 - val_loss: 0.3515 - val_accuracy: 0.8924


In [9]:
history = model.fit(x_train_normalized, y_train, epochs=2, batch_size=256, validation_data=(x_val_normalized, y_val))


Epoch 1/2
563/563 [==============================] - 46s 82ms/step - loss: 0.3270 - accuracy: 0.9005 - val_loss: 0.3239 - val_accuracy: 0.9003
Epoch 2/2
563/563 [==============================] - 44s 78ms/step - loss: 0.2908 - accuracy: 0.9116 - val_loss: 0.2895 - val_accuracy: 0.9107


In [11]:
model.save_weights('model/model.h5')

directory = "model"
# Save dictionary using pickle
with open(os.path.join(directory, 'label_mapping.pkl'), 'wb') as f:
    pickle.dump(label_mapping, f)
